## Transform Orders Data - Explode Arrays
1. Access elements from the JSON object
1. Deduplicate Array Elements
1. Explode Arrays
1. Write the Transformed Data to Silver Schema

In [0]:
SELECT * FROM gizmobox.silver.orders_json;

### 1. Access elements from the JSON object
`<column_name.object>`

In [0]:
SELECT json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       json_value.items
FROM gizmobox.silver.orders_json;

### 2. Deduplicate Array Elements
Function [array_distinct](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/array_distinct)

In [0]:
SELECT json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       array_distinct(json_value.items) AS items
FROM gizmobox.silver.orders_json;

### 3. Explode Arrays
Function [explode](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/explode)

In [0]:
CREATE OR REPLACE TEMP VIEW orders_exploded
AS
SELECT json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       explode(array_distinct(json_value.items)) AS item
FROM gizmobox.silver.orders_json;

In [0]:
SELECT 
  order_id,
  order_status,
  payment_method,
  total_amount,
  transaction_timestamp,
  customer_id,
  item.item_id,
  item.name,
  item.price,
  item.quantity,
  item.category,
  item.details.brand,
  item.details.color
FROM orders_exploded;

### 4. Write the Transformed Data to Silver Schema

In [0]:
CREATE TABLE IF NOT EXISTS gizmobox.silver.orders
AS
SELECT 
  order_id,
  order_status,
  payment_method,
  total_amount,
  transaction_timestamp,
  customer_id,
  item.item_id,
  item.name,
  item.price,
  item.quantity,
  item.category,
  item.details.brand,
  item.details.color
FROM orders_exploded;

In [0]:
SELECT * FROM gizmobox.silver.orders;